In [1]:
#Load our pretrained Model
#!pip install pyyaml h5py
import os
import tensorflow as tf
from keras.models import model_from_json
def normalize(img):
    img = img.astype("float32")
    min_perc, max_perc = np.percentile(img, 5), np.percentile(img, 95)
    img_valid = img[(img > min_perc) & (img < max_perc)]
    mean, std = img_valid.mean(), img_valid.std()
    img = (img-mean)/std
    return img
#!pip install h5
json_file = open(r'E:\CT\model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(r"E:\CT\model (1).h5")
import cv2
import numpy as np
unet=loaded_model



In [3]:
def filter_pixels(img):
    for i in range(img.shape[0]):
        for j in range(img.shape[3]):
            for k in range(img.shape[1]):
                for l in range(img.shape[2]):
                    img[i,k,l,j] = 1 if (img[i,k,l,j] > 0.5) else 0;
    return img

In [4]:
#Produce Masks
import SimpleITK as sitk
import os
for i in os.listdir(r"E:\\New Images"):
    print(i[0:-4])
#plt.imshow(pred_y[0,:,:,0])
    a=cv2.imread("E:\\New Images\\"+i)
    gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
    m=gray.flatten()
    gray=np.expand_dims(gray, axis=2)
    gray=np.expand_dims(gray, axis=0)
    m=normalize(gray)
    k=unet.predict(m)
    print(k.shape)
    gray=gray.reshape(512,512,1)
    pred_y = filter_pixels(k)
    pred_y=pred_y.reshape(512,512,4)
    sitk_image=sitk.GetImageFromArray(gray)
    sitk_mask = sitk.GetImageFromArray(pred_y[:,:,0])
    writer = sitk.ImageFileWriter()
    writer.SetFileName(os.path.join(r"E:\CT\Image Masks\PatientOrig",i[0:-4]+".nii"))
    writer.Execute(sitk_image)
    writer.SetFileName(os.path.join(r"E:\CT\Image Masks\Masks",i[0:-4]+"a"+".nii"))
    writer.Execute(sitk_mask)
    sitk_mask = sitk.GetImageFromArray(pred_y[:,:,1])
    writer.SetFileName(os.path.join(r"E:\CT\Image Masks\Masks",i[0:-4]+"b"+".nii"))
    writer.Execute(sitk_mask)
    sitk_mask = sitk.GetImageFromArray(pred_y[:,:,2])
    writer.SetFileName(os.path.join(r"E:\CT\Image Masks\Masks",i[0:-4]+"c"+".nii"))
    writer.Execute(sitk_mask)
    sitk_mask = sitk.GetImageFromArray(pred_y[:,:,3])
    writer.SetFileName(os.path.join(r"E:\CT\Image Masks\Masks",i[0:-4]+"d"+".nii"))
    writer.Execute(sitk_mask)
    

Patient101a
(1, 512, 512, 4)
Patient101b
(1, 512, 512, 4)
Patient101c
(1, 512, 512, 4)
Patient101d
(1, 512, 512, 4)
Patient103a
(1, 512, 512, 4)
Patient103b
(1, 512, 512, 4)
Patient103c
(1, 512, 512, 4)
Patient103d
(1, 512, 512, 4)
Patient104a
(1, 512, 512, 4)
Patient104b
(1, 512, 512, 4)
Patient104c
(1, 512, 512, 4)
Patient104d
(1, 512, 512, 4)
Patient106a
(1, 512, 512, 4)
Patient106b
(1, 512, 512, 4)
Patient106c
(1, 512, 512, 4)
Patient106d
(1, 512, 512, 4)
Patient107a
(1, 512, 512, 4)
Patient107b
(1, 512, 512, 4)
Patient107c
(1, 512, 512, 4)
Patient107d
(1, 512, 512, 4)
Patient108a
(1, 512, 512, 4)
Patient108b
(1, 512, 512, 4)
Patient108c
(1, 512, 512, 4)
Patient108d
(1, 512, 512, 4)
Patient110a
(1, 512, 512, 4)
Patient110b
(1, 512, 512, 4)
Patient110c
(1, 512, 512, 4)
Patient110d
(1, 512, 512, 4)
Patient112a
(1, 512, 512, 4)
Patient112b
(1, 512, 512, 4)
Patient112c
(1, 512, 512, 4)
Patient112d
(1, 512, 512, 4)
Patient116a
(1, 512, 512, 4)
Patient116b
(1, 512, 512, 4)
Patient116c
(1

In [44]:
#RADIOMICS
import pandas as pd
import numpy as np
from radiomics import featureextractor
import os
settings = {}
settings['correctMask'] = True
settings['geometryTolerance'] = 1
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
final=[]
finala = dict()
finalb = dict()
finalc = dict()
finald = dict()
count=1
path=r"E:\CT\Image Masks\PatientOrig"
for i in os.listdir(path):
  result=extractor.execute(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"a.nii"))
  finala["Patient"+i[7:-4]]=result[9:]
  result=extractor.execute(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"b.nii"))
  finalb["Patient"+i[7:-4]]=result[9:]
  result=extractor.execute(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"c.nii"))
  finalc["Patient"+i[7:-4]]=result[9:]
  result=extractor.execute(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"d.nii"))
  finald["Patient"+i[7:-4]]=result[9:]
df1=pd.DataFrame(finala)
print(df1.head())

Image/Mask geometry mismatch, attempting to correct Mask


RuntimeError: Exception thrown in SimpleITK Image_TransformPhysicalPointToContinuousIndex: d:\a\1\sitk\code\common\src\sitkPimpleImageBase.hxx:227:
sitk::ERROR: vector dimension mismatch

In [29]:
path=r"E:\CT\Image Masks\PatientOrig"
for i in os.listdir(path):
    print(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"a.nii"))
    print(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"b.nii"))
    print(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"c.nii"))
    print(os.path.join(path,i),os.path.join("E:\CT\Image Masks\Masks","Patient"+i[7:-4]+"d.nii"))


    
    

E:\CT\Image Masks\PatientOrig\Patient101a.nii E:\CT\Image Masks\Masks\Patient101aa.nii
E:\CT\Image Masks\PatientOrig\Patient101a.nii E:\CT\Image Masks\Masks\Patient101ab.nii
E:\CT\Image Masks\PatientOrig\Patient101a.nii E:\CT\Image Masks\Masks\Patient101ac.nii
E:\CT\Image Masks\PatientOrig\Patient101a.nii E:\CT\Image Masks\Masks\Patient101ad.nii
E:\CT\Image Masks\PatientOrig\Patient101b.nii E:\CT\Image Masks\Masks\Patient101ba.nii
E:\CT\Image Masks\PatientOrig\Patient101b.nii E:\CT\Image Masks\Masks\Patient101bb.nii
E:\CT\Image Masks\PatientOrig\Patient101b.nii E:\CT\Image Masks\Masks\Patient101bc.nii
E:\CT\Image Masks\PatientOrig\Patient101b.nii E:\CT\Image Masks\Masks\Patient101bd.nii
E:\CT\Image Masks\PatientOrig\Patient101c.nii E:\CT\Image Masks\Masks\Patient101ca.nii
E:\CT\Image Masks\PatientOrig\Patient101c.nii E:\CT\Image Masks\Masks\Patient101cb.nii
E:\CT\Image Masks\PatientOrig\Patient101c.nii E:\CT\Image Masks\Masks\Patient101cc.nii
E:\CT\Image Masks\PatientOrig\Patient101c.n

In [34]:
bg_data = sitk.GetArrayFromImage(sitk.ReadImage(r"E:\CT\Image Masks\Masks\Patient101aa.nii"))
bg_data.shape

(512, 512)

In [ ]:
dfa=pd.DataFrame.from_dict(finala, orient='index')
dfa.to_csv("typeAfeatures.csv")
dfa=pd.DataFrame.from_dict(finalb, orient='index')
dfa.to_csv("typeBfeatures.csv")
dfa=pd.DataFrame.from_dict(finalc, orient='index')
dfa.to_csv("typeBfeatures.csv")
dfa=pd.DataFrame.from_dict(finald, orient='index')
dfa.to_csv("typeDfeatures.csv")